In [1]:
import time
import cv2
cap = cv2.VideoCapture(0)

In [2]:
# Collect 1000 samples of your face from webcam input
count=0
while True:

    ret, face = cap.read()
    count += 1
    
    # Save file in specified directory with unique name
    file_name_path = 'C://Users//sumeet//Desktop//Mlops_task//dataset//Sumeet//sumeet' + str(count) + '.jpg'
    cv2.imwrite(file_name_path, face)

    # Put count on images and display live count
    cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Face', face)
    time.sleep(0.1)
        
    if cv2.waitKey(1) == 13 or count == 1000: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

In [3]:
from keras.applications import vgg16

In [4]:
# pre trained weights : create my model
model = vgg16.VGG16(weights='imagenet')

In [5]:
# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

#Loads the VGG16 model 
model = vgg16.VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

In [6]:
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [7]:
# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [8]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [9]:
model.input

<tf.Tensor 'input_2:0' shape=(None, 224, 224, 3) dtype=float32>

In [10]:
model.layers

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [12]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'C:/Users/sumeet/Desktop/Mlops_task/dataset/Train/'
validation_data_dir = 'C:/Users/sumeet/Desktop/Mlops_task/dataset/test/'


train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)


Found 1820 images belonging to 2 classes.
Found 180 images belonging to 2 classes.


In [25]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("face_vgg16.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1880
nb_validation_samples = 120
epochs = 4
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

modelnew.save("face_vgg16.h5")

Epoch 1/4
117/117 [==============================] - 408s 3s/step - loss: 0.7412 - accuracy: 0.8972 - val_loss: 4.9171e-04 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.00049, saving model to face_recog_vgg16.h5
Epoch 2/4
117/117 [==============================] - 417s 4s/step - loss: 0.1215 - accuracy: 0.9582 - val_loss: 0.0000e+00 - val_accuracy: 1.0000

Epoch 00002: val_loss improved from 0.00049 to 0.00000, saving model to face_recog_vgg16.h5
Epoch 3/4
117/117 [==============================] - 395s 3s/step - loss: 0.1263 - accuracy: 0.9647 - val_loss: 1.0514e-05 - val_accuracy: 1.0000

Epoch 00003: val_loss did not improve from 0.00000
Epoch 4/4
117/117 [==============================] - 387s 3s/step - loss: 0.0785 - accuracy: 0.9764 - val_loss: 0.0000e+00 - val_accuracy: 0.9857

Epoch 00004: val_loss did not improve from 0.00000


In [13]:
cap = cv2.VideoCapture(0)
status , photo = cap.read()
cv2.imshow('sumeet', photo)
cv2.imwrite('sumeet.jpg', photo)
cv2.waitKey()
cv2.destroyAllWindows()

In [14]:
from keras.preprocessing import image
img = image.load_img('sumeet.jpg', target_size=(224,224))

In [15]:
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

In [20]:
from keras.models import load_model
model_vgg16 = load_model('face_vgg16.h5')


In [21]:
output=model_vgg16.predict(img)
if(output[0][1]>output[0][0]):
    print("Sumeet")
else:
    print("Aditi")

Sumeet
